In [1]:
# 1. INITIAL SETUP & LIBRARIES
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import zipfile
import datetime
import json
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries and VGG16 modules imported successfully!")

Mounted at /content/drive
✅ Libraries and VGG16 modules imported successfully!


In [2]:
# 2. DATASET EXTRACTION
dataset_zip_path = '/content/drive/MyDrive/dataset.zip'
extract_folder = '/content/betel_dataset'

if not os.path.exists(dataset_zip_path):
    print(f"❌ Dataset not found at: {dataset_zip_path}")
else:
    if os.path.exists(extract_folder):
        shutil.rmtree(extract_folder)
    os.makedirs(extract_folder, exist_ok=True)
    print("📦 Extracting dataset...")
    with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
    print("✅ Dataset extracted successfully!")

📦 Extracting dataset...
✅ Dataset extracted successfully!


In [3]:
# 3. DIRECTORY SPLITTING (70/15/15)
base_path = '/content/betel_dataset'
source_data_path = os.path.join(base_path, 'dataset')
processed_data_root = os.path.join(base_path, 'commercial_classification')
os.makedirs(processed_data_root, exist_ok=True)

train_dir = os.path.join(processed_data_root, 'train')
val_dir = os.path.join(processed_data_root, 'val')
test_dir = os.path.join(processed_data_root, 'test')

# Logic to create splits if they don't exist
if not os.path.exists(train_dir):
    import random
    for split in ['train', 'val', 'test']:
        for cls in os.listdir(source_data_path):
            os.makedirs(os.path.join(processed_data_root, split, cls), exist_ok=True)

    for cls in os.listdir(source_data_path):
        cls_path = os.path.join(source_data_path, cls)
        if os.path.isdir(cls_path):
            images = [f for f in os.listdir(cls_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            random.shuffle(images)
            n = len(images)
            train_idx, val_idx = int(n*0.7), int(n*0.85)

            for i, img in enumerate(images):
                src = os.path.join(cls_path, img)
                if i < train_idx: dst = os.path.join(train_dir, cls, img)
                elif i < val_idx: dst = os.path.join(val_dir, cls, img)
                else: dst = os.path.join(test_dir, cls, img)
                shutil.copy(src, dst)
    print("✅ Training, Validation, and Test splits created!")

✅ Training, Validation, and Test splits created!


In [4]:
# 4. VGG16 DATA GENERATORS (With Heavy Augmentation)
# Note: VGG16 preprocess_input handles mean subtraction; we do NOT use rescale=1./255.
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    brightness_range=[0.7, 1.3],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=True)

val_generator = val_test_datagen.flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False)

test_generator = val_test_datagen.flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False)

num_classes = len(train_generator.class_indices)

Found 1131 images belonging to 4 classes.
Found 242 images belonging to 4 classes.
Found 245 images belonging to 4 classes.


In [15]:
# 5. BUILDING THE MODEL (FIXED: Added layers before summary)
print("\n🏗️ Building VGG16 Model...")
# Load the base
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg_base.trainable = False # Freeze base initially

# Create the Sequential model and ADD layers
model = models.Sequential()
model.add(vgg_base)                 # Add base model first
model.add(layers.Flatten())         # Convert 3D features to 1D
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))      # Regularization to prevent overfitting
model.add(layers.Dense(num_classes, activation='softmax'))

# Now summary will show all parameters
model.summary()


🏗️ Building VGG16 Model...


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,138,500 (80.64 MB)

 Trainable params: 6,423,812 (24.50 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [24]:
# Use a low learning rate for Phase 1 to protect the pre-trained base
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

save_dir = '/content/vgg16_checkpoints'
os.makedirs(save_dir, exist_ok=True)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Properly formatted callback list to prevent syntax errors
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(save_dir, f'vgg16_best_{timestamp}.h5'),
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

print("✅ Callbacks and compilation configured successfully!")

✅ Callbacks and compilation configured successfully!


In [25]:
print("\n🚀 TRAINING PHASE 1: Feature Extraction (Base Frozen)...")
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=30,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)


🚀 TRAINING PHASE 1: Feature Extraction (Base Frozen)...
Epoch 1/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8465 - loss: 0.4253
Epoch 1: val_accuracy improved from -inf to 0.84821, saving model to /content/vgg16_checkpoints/vgg16_best_20260102_163841.h5


35/35 ━━━━━━━━━━━━━━━━━━━━ 107s 3s/step - accuracy: 0.8466 - loss: 0.4265 - val_accuracy: 0.8482 - val_loss: 0.5746
Epoch 2/30
 1/35 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8125 - loss: 0.4861
Epoch 2: val_accuracy did not improve from 0.84821
35/35 ━━━━━━━━━━━━━━━━━━━━ 14s 404ms/step - accuracy: 0.8125 - loss: 0.4861 - val_accuracy: 0.8438 - val_loss: 0.5803
Epoch 3/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8548 - loss: 0.4613
Epoch 3: val_accuracy improved from 0.84821 to 0.87054, saving model to /content/vgg16_checkpoints/vgg16_best_20260102_163841.h5


35/35 ━━━━━━━━━━━━━━━━━━━━ 98s 3s/step - accuracy: 0.8549 - loss: 0.4604 - val_accuracy: 0.8705 - val_loss: 0.4913
Epoch 4/30
 1/35 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8750 - loss: 0.3821
Epoch 4: val_accuracy did not improve from 0.87054
35/35 ━━━━━━━━━━━━━━━━━━━━ 13s 384ms/step - accuracy: 0.8750 - loss: 0.3821 - val_accuracy: 0.8661 - val_loss: 0.4947
Epoch 5/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8536 - loss: 0.4634
Epoch 5: val_accuracy did not improve from 0.87054
35/35 ━━━━━━━━━━━━━━━━━━━━ 92s 3s/step - accuracy: 0.8535 - loss: 0.4630 - val_accuracy: 0.8036 - val_loss: 0.7998
Epoch 6/30
 1/35 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7188 - loss: 0.8958
Epoch 6: val_accuracy did not improve from 0.87054
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 458ms/step - accuracy: 0.7188 - loss: 0.8958 - val_accuracy: 0.8214 - val_loss: 0.7709
Epoch 7/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8490 - loss: 0.4650
Epoch 7: val_accuracy did not improve from 0.

35/35 ━━━━━━━━━━━━━━━━━━━━ 98s 3s/step - accuracy: 0.8770 - loss: 0.3191 - val_accuracy: 0.8839 - val_loss: 0.3857
Epoch 14/30
 1/35 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.8438 - loss: 0.3808
Epoch 14: val_accuracy did not improve from 0.88393
35/35 ━━━━━━━━━━━━━━━━━━━━ 15s 443ms/step - accuracy: 0.8438 - loss: 0.3808 - val_accuracy: 0.8839 - val_loss: 0.3960
Epoch 15/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8819 - loss: 0.3495
Epoch 15: val_accuracy did not improve from 0.88393
35/35 ━━━━━━━━━━━━━━━━━━━━ 97s 3s/step - accuracy: 0.8816 - loss: 0.3498 - val_accuracy: 0.8705 - val_loss: 0.4626
Epoch 16/30
 1/35 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8750 - loss: 0.1412
Epoch 16: val_accuracy did not improve from 0.88393
35/35 ━━━━━━━━━━━━━━━━━━━━ 15s 454ms/step - accuracy: 0.8750 - loss: 0.1412 - val_accuracy: 0.8616 - val_loss: 0.4765
Epoch 17/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8738 - loss: 0.3705
Epoch 17: val_accuracy did not improve

35/35 ━━━━━━━━━━━━━━━━━━━━ 92s 3s/step - accuracy: 0.8663 - loss: 0.3485 - val_accuracy: 0.8973 - val_loss: 0.3426
Epoch 30/30
 1/35 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8438 - loss: 0.3215
Epoch 30: val_accuracy did not improve from 0.89732
35/35 ━━━━━━━━━━━━━━━━━━━━ 15s 455ms/step - accuracy: 0.8438 - loss: 0.3215 - val_accuracy: 0.8929 - val_loss: 0.3427


In [26]:
print("\n🚀 TRAINING PHASE 1: Feature Extraction (Base Frozen)...")
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=30,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)


🚀 TRAINING PHASE 1: Feature Extraction (Base Frozen)...
Epoch 1/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8778 - loss: 0.3281
Epoch 1: val_accuracy did not improve from 0.89732
35/35 ━━━━━━━━━━━━━━━━━━━━ 94s 3s/step - accuracy: 0.8779 - loss: 0.3284 - val_accuracy: 0.8795 - val_loss: 0.3624
Epoch 2/30
 1/35 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8438 - loss: 0.5047
Epoch 2: val_accuracy did not improve from 0.89732
35/35 ━━━━━━━━━━━━━━━━━━━━ 15s 442ms/step - accuracy: 0.8438 - loss: 0.5047 - val_accuracy: 0.8795 - val_loss: 0.3669
Epoch 3/30
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8898 - loss: 0.2857
Epoch 3: val_accuracy did not improve from 0.89732
35/35 ━━━━━━━━━━━━━━━━━━━━ 89s 3s/step - accuracy: 0.8901 - loss: 0.2857 - val_accuracy: 0.8571 - val_loss: 0.4103
Epoch 4/30
 1/35 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9375 - loss: 0.1768
Epoch 4: val_accuracy did not improve from 0.89732
35/35 ━━━━━━━━━━━━━━━━━━━━ 15s 447ms/step - accuracy: 0

In [27]:
print("\n🎯 Final Test Set Performance Evaluation:")
# Load the best weights saved by the checkpoint before evaluating
model.load_weights(os.path.join(save_dir, f'vgg16_best_{timestamp}.h5'))

test_loss, test_acc = model.evaluate(test_generator)
print(f"\n✅ FINAL TEST ACCURACY: {test_acc*100:.2f}%")
print(f"✅ FINAL TEST LOSS: {test_loss:.4f}")


🎯 Final Test Set Performance Evaluation:
8/8 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.9260 - loss: 0.1690

✅ FINAL TEST ACCURACY: 86.53%
✅ FINAL TEST LOSS: 0.3396
